In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from konlpy.tag import Kkma
tagger= Kkma()

## 1. RNNCell 

$$h′=tanh(wih∗x+bih+whh∗h+bhh)$$
class torch.nn.RNNCell(input_size, hidden_size, bias=True, nonlinearity='tanh')


In [5]:
input_size = 7 # input dimension (word embedding) D
hidden_size = 30 # hidden dimension H
batch_size = 3
length = 7

In [6]:
rnncell = nn.RNNCell(input_size=input_size,
                               hidden_size=hidden_size,
                               bias=True, 
                               nonlinearity='tanh')

In [7]:
rnncell

RNNCell(7, 30)

In [8]:
input = Variable(torch.randn(length, batch_size, input_size)) # T, B, D    (embedding matrix에서 인덱싱한 워드 벡터)
hidden = Variable(torch.zeros(batch_size, hidden_size)) # first hidden state
output = []
for i in range(length):
    hidden = rnncell(input[i], hidden)
    output.append(hidden)

In [9]:
hidden.size() # Batch size, Hidden_size

torch.Size([3, 30])

In [12]:
output

[tensor([[-0.1387, -0.1960, -0.1948, -0.4071, -0.0709, -0.1428,  0.0559,
          -0.0700, -0.0183,  0.1165, -0.5116,  0.0152, -0.3726,  0.1087,
          -0.1083, -0.0054,  0.2544,  0.2849, -0.2834, -0.0698,  0.2597,
          -0.0167, -0.4322,  0.0872,  0.0709,  0.4363,  0.5371,  0.3305,
          -0.3622, -0.0763],
         [ 0.2018,  0.0750,  0.0865,  0.2297,  0.1160, -0.2435,  0.3663,
          -0.2089, -0.2707,  0.0472,  0.1430,  0.3238, -0.2474, -0.0757,
           0.4768,  0.0509, -0.0355,  0.0320,  0.1231, -0.2651, -0.0207,
          -0.0577, -0.2245, -0.2263,  0.0323,  0.5218,  0.1249,  0.2893,
          -0.1953, -0.1859],
         [-0.1803, -0.1973, -0.0838, -0.0376, -0.1198, -0.2727, -0.0943,
          -0.4005,  0.0620, -0.3936, -0.1270,  0.5793, -0.3176, -0.5552,
           0.3895, -0.3903, -0.0843,  0.0834,  0.0636, -0.3057,  0.1149,
          -0.1081, -0.3499, -0.4602,  0.3169,  0.3554,  0.1644,  0.2450,
          -0.0294, -0.3000]]),
 tensor([[-0.2224, -0.0178, -0.3118

## 2. RNN

## class torch.nn.RNN(*args, **kwargs)
### Parameters:	
 - **input_size** – The number of expected features in the input x
 - **hidden_size** – The number of features in the hidden state h
 - **num_layers** – Number of recurrent layers. E.g., setting num_layers=2 would mean stacking two RNNs together to form a stacked RNN, with the second RNN taking in outputs of the first RNN and computing the final results. Default: 1
 - **nonlinearity** – The non-linearity to use. Can be either ‘tanh’ or ‘relu’. Default: ‘tanh’
 - **bias** – If False, then the layer does not use bias weights b_ih and b_hh. Default: True
 - **batch_first** – If True, then the input and output tensors are provided as (batch, seq, feature)
 - **dropout** – If non-zero, introduces a Dropout layer on the outputs of each RNN layer except the last layer, with dropout probability equal to dropout. Default: 0
 - **bidirectional** – If True, becomes a bidirectional RNN. Default: False

In [22]:
input_size = 7
hidden_size = 30
batch_size = 16
length = 4
output_size = 7

rnn = nn.RNN(input_size, hidden_size, batch_first=True) 
#,num_layers=1,bias=True,nonlinearity='tanh', batch_first=True, dropout=0, bidirectional=False)

# (num_layers * num_directions, batch, hidden_size)
input = Variable(torch.randn(batch_size, length,input_size)) # B,T,D  <= batch_first
hidden = Variable(torch.zeros(1, batch_size, hidden_size)) # 1,B,H    (num_layers * num_directions, batch, hidden_size)

output, hidden = rnn(input, hidden)

In [23]:
output.size() # B,T,H

torch.Size([16, 4, 30])

In [24]:
hidden.size() # 1,B,H

torch.Size([1, 16, 30])

In [25]:
linear = nn.Linear(hidden_size,output_size)
output = F.softmax(linear(output),1)
output.size()

torch.Size([16, 4, 7])

## 3. Bidirectional RNN 

In [26]:
rnn = nn.RNN(input_size, hidden_size,num_layers=1,bias=True,nonlinearity='tanh', batch_first=True, dropout=0, bidirectional=True)

In [27]:
input = Variable(torch.randn(batch_size,length,input_size)) # B,T,D
hidden = Variable(torch.zeros(2,batch_size,hidden_size)) # 2,B,H    (num_layers * num_directions, batch, hidden_size)

In [28]:
output, hidden = rnn(input,hidden)

In [29]:
output.size() # concat of forward,backward

torch.Size([16, 4, 60])

In [30]:
hidden.size() # forward, backward

torch.Size([2, 16, 30])

## 4. Multi-layer RNN 

In [31]:
input_size = 10
hidden_size = 30
batch_size = 3
length = 4
output_size = 5
rnn = nn.RNN(input_size, 
             hidden_size, 
             num_layers=3,
             bias=True, 
             nonlinearity='tanh', 
             batch_first=True, 
             dropout=0, bidirectional=True)

In [32]:
input = Variable(torch.randn(batch_size,length,input_size)) # B,T,D
hidden = Variable(torch.zeros(3*2,batch_size,hidden_size)) # 6,B,H    (num_layers * num_directions, batch, hidden_size)

In [33]:
output, hidden = rnn(input,hidden)

In [34]:
output.size()

torch.Size([3, 4, 60])

In [35]:
hidden.size() # (forward, backward)*num_layers

torch.Size([6, 3, 30])

## 5. GRU 

In [36]:
rnn = nn.GRU(input_size,hidden_size,batch_first=True) #,num_layers=1,bias=True,batch_first=True,bidirectional=True)

In [37]:
input = Variable(torch.randn(batch_size,length,input_size)) # B,T,D
hidden = Variable(torch.zeros(1,batch_size,hidden_size)) # 2,B,H

In [38]:
output, hidden = rnn(input,hidden)

In [39]:
print(output.size())
print(hidden.size())

torch.Size([3, 4, 30])
torch.Size([1, 3, 30])


## 6. LSTM
<br/>

Parameters:	<br/>
- input_size – The number of expected features in the input x<br/>
- hidden_size – The number of features in the hidden state h<br/>
- num_layers – Number of recurrent layers. E.g., setting num_layers=2 would mean stacking two LSTMs together to form a stacked LSTM, with the second LSTM taking in outputs of the first LSTM and computing the final results. Default: 1<br/>
- bias – If False, then the layer does not use bias weights b_ih and b_hh. Default: True<br/>
- batch_first – If True, then the input and output tensors are provided as (batch, seq, feature)<br/>
- dropout – If non-zero, introduces a Dropout layer on the outputs of each LSTM layer except the last layer, with dropout probability equal to dropout. Default: 0<br/>
- bidirectional – If True, becomes a bidirectional LSTM. Default: False<br/>

>Inputs: input, (h_0, c_0)<br/>
 - input of shape (seq_len, batch, input_size): tensor containing the features of the input sequence. The input can also be a packed variable length sequence. See torch.nn.utils.rnn.pack_padded_sequence() or torch.nn.utils.rnn.pack_sequence() for details.
 - h_0 of shape (num_layers * num_directions, batch, hidden_size): tensor containing the initial hidden state for each element in the batch.
 - c_0 of shape (num_layers * num_directions, batch, hidden_size): tensor containing the initial cell state for each element in the batch.
 - If (h_0, c_0) is not provided, both h_0 and c_0 default to zero.

> Outputs: output, (h_n, c_n)<br/>
 - output of shape (seq_len, batch, hidden_size * num_directions): tensor containing the output features (h_t) from the last layer of the LSTM, for each t. If a torch.nn.utils.rnn.PackedSequence has been given as the input, the output will also be a packed sequence.
 - h_n of shape (num_layers * num_directions, batch, hidden_size): tensor containing the hidden state for t = seq_len
 - c_n (num_layers * num_directions, batch, hidden_size): tensor containing the cell state for t = seq_len

In [40]:
rnn = nn.LSTM(10, 20, 2)
input = Variable(torch.randn(5, 3, 10))
h0 = Variable(torch.randn(2, 3, 20))
c0 = Variable(torch.randn(2, 3, 20))
output, hn = rnn(input, (h0, c0))

In [41]:
input_size = 10
hidden_size = 30
output_size = 10
batch_size = 3
length = 4
num_layers = 3

In [42]:
rnn = nn.LSTM(input_size,
              hidden_size,
              num_layers=num_layers,
              bias=True,
              batch_first=True,
              bidirectional=True)

In [43]:
input = Variable(torch.randn(batch_size,length,input_size)) # B,T,D
hidden = Variable(torch.zeros(num_layers*2,batch_size,hidden_size)) # (num_layers * num_directions, batch, hidden_size)
cell = Variable(torch.zeros(num_layers*2,batch_size,hidden_size)) # (num_layers * num_directions, batch, hidden_size)

In [44]:
output, (hidden,cell) = rnn(input,(hidden,cell))

In [45]:
print(input.size())
print(output.size())
print(hidden.size())
print(cell.size())

torch.Size([3, 4, 10])
torch.Size([3, 4, 60])
torch.Size([6, 3, 30])
torch.Size([6, 3, 30])


In [46]:
linear = nn.Linear(hidden_size*2, output_size)
output = F.softmax(linear(output),1)
output.size()

torch.Size([3, 4, 10])

## TODO 각 timestep마다 그 다음에 올 단어를 예측하는 Language model을 만드시오 

* 다음 Corpus(sentences)를 tokenized하고 Vocab을 만드시오
* Embedding matrix(vector size는 10)
* hidden state의 size가 20인 Bidirectional GRU(num_layers=1)

In [47]:
from konlpy.tag import Kkma
tagger = Kkma()

In [48]:
sentences=["나는 오늘 삼계탕을 먹었다","그런데도 배가 아직 고프다","이제 영화보러 가야겠다 요즘 뭐가 재밌지","날씨가 진짜 좋다"]

In [49]:
tokenized = [tagger.morphs(s) for s in sentences]
vocab = list((set([token for tokens in tokenized for token in tokens])))
word2index = {v:i for i,v in enumerate(vocab)}

In [50]:
len(word2index)

29

In [51]:
V = len(word2index)
D = 10
H = 20

In [52]:
class LM(nn.Module):
    def __init__(self,V,D,H):
        super(LM,self).__init__()
        self.hidden_size = H
        
        self.embed = nn.Embedding(V,D) # VxD
        self.gru = nn.GRU(D,H,1,batch_first=True,bidirectional=True)
        self.linear = nn.Linear(H*2,V)
    
    def init_hidden(self,batch_size):
        hidden = Variable(torch.zeros(2,batch_size,self.hidden_size))
        return hidden
    
    def forward(self,inputs):
        """
        inputs : B,T # LongTensor
        """
        embed = self.embed(inputs) # B,T,D
        hidden = self.init_hidden(inputs.size(0)) # 2,B,H
        output, hidden = self.gru(embed,hidden)
        # output : B,T,2H
        # hidden : 2,B,H
        
        output = self.linear(output) # B,T,V
        return output

In [53]:
model = LM(len(word2index),10,20)

In [54]:
model

LM(
  (embed): Embedding(29, 10)
  (gru): GRU(10, 20, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=40, out_features=29, bias=True)
)

## 7. Standard form 

In [55]:
class RNN(nn.Module):
    def __init__(self,input_size,embed_size,hidden_size,output_size,num_layers=1,bidirec=False):
        super(RNN,self).__init__()
        
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        if bidirec:
            self.num_directions = 2
        else:
            self.num_directions = 1
            
        self.embed = nn.Embedding(input_size,embed_size)
        self.lstm = nn.LSTM(embed_size,hidden_size,num_layers,batch_first=True,bidirectional=bidirec)
        self.linear = nn.Linear(hidden_size*self.num_directions,output_size)
        
    def init_hidden(self,batch_size):
        # (num_layers * num_directions, batch_size, hidden_size)
        hidden = Variable(torch.zeros(self.num_layers*self.num_directions,batch_size,self.hidden_size))
        cell = Variable(torch.zeros(self.num_layers*self.num_directions,batch_size,self.hidden_size))
        return hidden, cell
    
    def forward(self,inputs):
        """
        inputs : B,T
        """
        embed = self.embed(inputs) # word vector indexing
        hidden, cell = self.init_hidden(inputs.size(0)) # initial hidden,cell
        
        output, (hidden,cell) = self.lstm(embed,(hidden,cell))
        
        # Many-to-Many
        output = self.linear(output) # B,T,H -> B,T,V
        
        # Many-to-One
        #hidden = hidden[-self.num_directions:] # (num_directions,B,H)
        #hidden = torch.cat([h for h in hidden],1)
        #output = self.linear(hidden) # last hidden
        
        return output

In [56]:
VOCAB=1000 # input_size
EMBED = 50 # embedding_size
HIDDEN = 100 # hidden_size

In [57]:
rnn = RNN(VOCAB,EMBED,HIDDEN,VOCAB,bidirec=True)

In [58]:
test_input = Variable(torch.randperm(32*10)).view(32,10)

In [59]:
test_input # 길이 10개짜리 문장의 32개 배치

tensor([[  20,  180,  179,  244,   29,  252,   42,  215,   59,  133],
        [  85,  115,    0,   57,  271,  114,  318,    3,   56,  232],
        [  11,   25,  309,   77,  146,  248,  296,  316,  313,  169],
        [  19,   99,   24,  144,  308,  226,  223,  112,   14,   47],
        [ 219,  243,  293,  303,  267,    8,   71,  294,  314,  229],
        [ 317,  270,   39,  312,  105,  282,  305,   86,  284,  217],
        [  68,  290,  150,  192,  272,  222,  273,   30,  258,  203],
        [ 182,  235,  137,  113,  278,  186,  281,   40,  121,  299],
        [  52,   96,  122,  165,  297,  246,   49,   12,  153,  175],
        [ 206,  190,  170,  106,    6,  191,  279,  197,   92,    4],
        [  81,  231,   23,  151,   50,  157,  210,  136,  208,    7],
        [ 266,  125,   84,  176,  111,   48,  131,  301,  311,  289],
        [ 204,   17,  315,   31,   18,  227,  236,   10,  103,  164],
        [  98,  163,  156,  195,  225,  260,  220,   75,  127,  245],
        [ 241,    2,

In [60]:
output = rnn(test_input)

In [61]:
output.size()

torch.Size([32, 10, 1000])

## TODO : Sentence Classifier

* data/train.txt를 torchtext로 load하시오(배치사이즈 5)
* Bidirectional LSTM을 선언(num_layers=2)
* 마지막 히든 스테이트를 이용하여 Binary Classifier를 만드시오(Many-to-One)
* train 시키시오

In [62]:
from torchtext.data import Field,Iterator,TabularDataset

In [63]:
TEXT = Field(tokenize=tagger.morphs,use_vocab=True, batch_first=True)
LABEL = Field(sequential=False,use_vocab=True,unk_token=None)

train_data = TabularDataset(path="data/train.txt",
                            format="tsv",
                            fields=[('TEXT',TEXT),('LABEL',LABEL)])

In [64]:
print(train_data.examples[0].TEXT)
print(train_data.examples[0].LABEL)

['[', '"', '배고프', '다', '밥', '주', '어', '"', ',', '"', 'FOOD', '"', ']', ',']


AttributeError: 'Example' object has no attribute 'LABEL'

In [ ]:
TEXT.build_vocab(train_data)
LABEL.build_vocab(train_data)

In [ ]:
train_loader =  Iterator(train_data, batch_size=5, device=-1, # device -1 : cpu, device 0 : 남는 gpu
    sort_key=lambda x: len(x.TEXT),sort_within_batch=True,repeat=False) # x.TEXT 길이 기준으로 정렬

In [ ]:
class RNN(nn.Module):
    def __init__(self,input_size,embed_size,hidden_size,output_size,num_layers=1,bidirec=False):
        super(RNN,self).__init__()
        
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        if bidirec:
            self.num_directions = 2
        else:
            self.num_directions = 1
            
        self.embed = nn.Embedding(input_size,embed_size)
        self.lstm = nn.LSTM(embed_size,hidden_size,num_layers,batch_first=True,bidirectional=bidirec)
        self.linear = nn.Linear(hidden_size*self.num_directions,output_size)
        
    def init_hidden(self,batch_size):
        # (num_layers * num_directions, batch_size, hidden_size)
        hidden = Variable(torch.zeros(self.num_layers*self.num_directions,batch_size,self.hidden_size))
        cell = Variable(torch.zeros(self.num_layers*self.num_directions,batch_size,self.hidden_size))
        return hidden, cell
    
    def forward(self,inputs):
        """
        inputs : B,T
        """
        embed = self.embed(inputs) # word vector indexing
        hidden, cell = self.init_hidden(inputs.size(0)) # initial hidden,cell
        
        output, (hidden,cell) = self.lstm(embed,(hidden,cell))
        
        # Many-to-One
        hidden = hidden[-self.num_directions:] # (num_directions,B,H)
        hidden = torch.cat([h for h in hidden],1)
        output = self.linear(hidden) # last hidden
        
        return output

In [ ]:
model = RNN(len(TEXT.vocab),30,50,len(LABEL.vocab),bidirec=True)
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=0.001)

In [ ]:
for batch in train_loader:
    model.zero_grad()
    preds = model(batch.TEXT)
    loss = loss_function(preds,batch.LABEL)
    print(loss.data[0])
    loss.backward()
    optimizer.step()